In [1]:
import numpy as np
import librosa
import os
import pandas as pd
from os import walk
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

In [2]:
def get_chroma_vector(wav_file_path, target_duration=3):
    y, sr = librosa.load(wav_file_path, sr=None)
    current_duration = librosa.get_duration(y=y, sr=sr)
    
    # Calculez le facteur d'étirement ou de compression nécessaire
    time_stretch_factor = current_duration / target_duration
    # Étirez ou compressez le signal audio pour atteindre la durée cible
    y_stretched = librosa.effects.time_stretch(y,rate=time_stretch_factor)
    # Calculez le chromagramme à partir du signal modifié
    chroma = np.array(librosa.feature.chroma_stft(y=y_stretched, sr=sr))
    df_chroma = pd.DataFrame(chroma)
    moyenne_par_colonne = df_chroma.mean()
    # Sélectionnez les colonnes dont la moyenne est inférieure ou égale à 0.6
    colonnes_a_conserver = moyenne_par_colonne[moyenne_par_colonne <= 0.65].index

    # Créez un nouveau DataFrame en ne conservant que les colonnes sélectionnées
    df_filtered = df_chroma[colonnes_a_conserver]
    new_chroma = df_filtered.to_numpy()
    return new_chroma


In [6]:
guitar_file = r"Chords_generation\chords_wav\F_A#aug.wav"
flute_file = r"Chords_generation\chords_wav\F_A#aug.wav"
piano_file = r"\Chords_generation\chords_wav\F_A#aug.wav"
violin_file = r"Chords_generation\chords_wav\F_A#aug.wav"

In [7]:

# Obtenez les 4 chromagrammes (remplacez ces lignes par les données réelles que vous avez)
chroma1 = get_chroma_vector(flute_file)
chroma2 = get_chroma_vector(guitar_file)
chroma3 = get_chroma_vector(piano_file)
chroma4 = get_chroma_vector(violin_file)

# Créez une figure avec une grille de 2x2 sous-plots
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Affichez chaque chromagramme dans un sous-plot
img1 = librosa.display.specshow(chroma1, y_axis='chroma', x_axis='time', ax=axes[0, 0])
axes[0, 0].set_title('Flute Chroma')

img2 = librosa.display.specshow(chroma2, y_axis='chroma', x_axis='time', ax=axes[0, 1])
axes[0, 1].set_title('Guitar Chroma')

img3 = librosa.display.specshow(chroma3, y_axis='chroma', x_axis='time', ax=axes[1, 0])
axes[1, 0].set_title('Piano Chroma')

img4 = librosa.display.specshow(chroma4, y_axis='chroma', x_axis='time', ax=axes[1, 1])
axes[1, 1].set_title('Violin Chroma')

# Ajoutez une barre de couleur pour chaque sous-plot
fig.colorbar(img1, ax=axes[0, 0])
fig.colorbar(img2, ax=axes[0, 1])
fig.colorbar(img3, ax=axes[1, 0])
fig.colorbar(img4, ax=axes[1, 1])

# Ajustez l'espacement entre les sous-plots
plt.tight_layout()

# Affichez la figure
plt.show()


C:\Users\zalat\AppData\Local\Temp\ipykernel_20204\1701673794.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(wav_file_path, sr=None)


FileNotFoundError: [Errno 2] No such file or directory: 'Chords_generation\\chords_wav\\F_A#aug.wav'

In [ ]:
def get_feature(file_path):
 # Extracting chroma vector feature
  chroma = get_chroma_vector(file_path)
  chroma_mean = chroma.mean(axis=1)

  feature = chroma_mean
  return feature

In [ ]:
names = ['C', 'C#', 'D','D#','E','F','F#','G','G#','A','A#','B'] # nom des barres

values_f = get_feature(flute_file).tolist()
values_g = get_feature(guitar_file).tolist()
values_p = get_feature(piano_file).tolist()
values_v = get_feature(violin_file).tolist()

plt.subplot(221)
plt.bar(names, values_f)
plt.text(0.5, 0.5, "F_Gmaj.wav", size=12, alpha=.5)

plt.subplot(222)
plt.bar(names, values_g)
plt.text(0.5,0.5, "G_Gmaj.wav.wav", size=12, alpha=.5)

plt.subplot(223)
plt.bar(names, values_p)
plt.text(0.5,0.5, "P_Gmaj.wav.wav", size=12, alpha=.5)

plt.subplot(224)
plt.bar(names, values_v)
plt.text(0.5,0.5, "V_Gmaj.wav", size=12, alpha=.5)

plt.show()
